# Asymmetry Detection Models
This notebook shows the process of load and train of a LSTM neuronal network to classify the asymmetry laterality of a PD patient or a control. In this notebook
we will show different implementations of LSTMs (Normal and bidirectional) and with different hyperparameters.

## Requiered Imports

In [1]:
import pickle

import tensorflow as tf
import numpy as np

from keras_tuner.tuners import RandomSearch


from src.model.asa_models import build_basic_lstm, build_2layer_lstm, build_1d_conv_1lstm, build_1d_conv_1layer_lstm_do

from src.settings import ROOT_DIR
from src.utils.data_split import get_features_target
from src.model.callbacks_builder import get_callbacks, build_tensorboard

## Data load

In [2]:
train_data_path = ROOT_DIR / 'data' / 'processed' / 'asa' / 'train.pkl'
val_data_path = ROOT_DIR / 'data' / 'processed' / 'asa' / 'val.pkl'

with open(train_data_path, 'rb') as file:
    train_data = pickle.load(file)

with open(val_data_path, 'rb') as file:
    val_data = pickle.load(file)

In [3]:
len(train_data)

124

In [4]:
X_train, y_train = get_features_target(train_data)
X_val, y_val = get_features_target(val_data)

In [5]:
max_length = max(len(x) for x in X_train)
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_length, padding='post', dtype='float32')

max_length = max(len(x) for x in X_val)
X_val_padded = tf.keras.preprocessing.sequence.pad_sequences(X_val, maxlen=max_length, padding='post', dtype='float32')

In [6]:
y_train = np.array(y_train)
y_val = np.array(y_val)

## Tensor Board
This tool will help us to supervise the experimentation process of all the experimental models described in this notebook

In [7]:
%load_ext tensorboard
%tensorboard --logdir ../results/asa/lightning_logs/

Reusing TensorBoard on port 6006 (pid 15016), started 1:16:52 ago. (Use '!kill 15016' to kill it.)

In [8]:
tensorboard_callback = build_tensorboard('asa')

## Models
Every model will be trained looking for the best hyperparameters, for this we will use Random Search.

In [9]:
RS_EPOCHS = 20

### Simple LSTM
This model is a 64-lstm with any hidden layers and just one dense layer to infer the output.
Optimizer: *Adam*
Loss: *binary_crossentropy*

In [10]:
BATCH_SIZE = 14
EPOCHS = 200

In [11]:
tunner_model = RandomSearch(
    build_basic_lstm,
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=1
)

In [18]:
tunner_model.search(X_train_padded, y_train,
                    epochs=RS_EPOCHS,
                    validation_data=(X_val_padded, y_val)
                    )

Trial 4 Complete [00h 00m 48s]
val_accuracy: 0.7200000286102295

Best val_accuracy So Far: 0.7200000286102295
Total elapsed time: 00h 04m 24s


In [19]:
best_hyperparams = tunner_model.get_best_hyperparameters(1)[0]
model = tunner_model.hypermodel.build(best_hyperparams)

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128)               78336     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 78465 (306.50 KB)
Trainable params: 78465 (306.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
checkpoint_model, early_stop_model = get_callbacks('asa', '64-lstm-no-hidden')

In [23]:
# Train the model
history_model = model.fit(
    X_train_padded,
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val_padded, y_val),
    callbacks=[tensorboard_callback, checkpoint_model, early_stop_model])

Epoch 1/200
9/9 [==============================] - ETA: 0s - loss: 0.7059 - accuracy: 0.6694
Epoch 1: val_accuracy improved from -inf to 0.72000, saving model to /content/AIForIMUParkinsonData/results/asa/64-lstm-no-hidden/tmp/ckpt/checkpoint.model.keras
9/9 [==============================] - 8s 600ms/step - loss: 0.7059 - accuracy: 0.6694 - val_loss: 0.6303 - val_accuracy: 0.7200
Epoch 2/200
9/9 [==============================] - ETA: 0s - loss: 0.6702 - accuracy: 0.6694
Epoch 2: val_accuracy did not improve from 0.72000
9/9 [==============================] - 5s 555ms/step - loss: 0.6702 - accuracy: 0.6694 - val_loss: 0.6292 - val_accuracy: 0.7200
Epoch 3/200
9/9 [==============================] - ETA: 0s - loss: 0.6520 - accuracy: 0.6694
Epoch 3: val_accuracy did not improve from 0.72000
9/9 [==============================] - 5s 582ms/step - loss: 0.6520 - accuracy: 0.6694 - val_loss: 0.5975 - val_accuracy: 0.7200
Epoch 4/200
9/9 [==============================] - ETA: 0s - loss: 0.6

### 1 Hidden layers LSTM

In [ ]:
BATCH_SIZE = 14
EPOCHS = 150

In [0]:
tunner_lstm = RandomSearch(
    build_2layer_lstm,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1
)

Reloading Tuner from ./untitled_project/tuner0.json


In [0]:
tunner_lstm.search(X_train_padded, y_train,
                    epochs=RS_EPOCHS,
                    validation_data=(X_val_padded, y_val)
                    )

Trial 10 Complete [00h 02m 26s]
val_accuracy: 0.7200000286102295

Best val_accuracy So Far: 0.7200000286102295
Total elapsed time: 00h 30m 16s


In [0]:
best_hyperparams_lstm = tunner_lstm.get_best_hyperparameters(1)[0]
lstm = tunner_lstm.hypermodel.build(best_hyperparams_lstm)

In [0]:
tunner_lstm.get_best_hyperparameters(1)

In [0]:
lstm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 16723, 128)        78336     
                                                                 
 lstm_3 (LSTM)               (None, 192)               246528    
                                                                 
 dense_1 (Dense)             (None, 1)                 193       
                                                                 
Total params: 325057 (1.24 MB)
Trainable params: 325057 (1.24 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [0]:
checkpoint_lstm, early_stop_lstm = get_callbacks('asa', 'lstm-1hidden')

In [0]:
history_lstm = lstm.fit(x=X_train_padded,
                        y=y_train,
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=(X_val_padded, y_val),
                        callbacks=[tensorboard_callback, checkpoint_lstm, early_stop_lstm])

Epoch 1/150
9/9 [==============================] - ETA: 0s - loss: 1.0994 - accuracy: 0.6694
Epoch 1: val_accuracy improved from -inf to 0.72000, saving model to /content/AIForIMUParkinsonData/results/asa/lstm-1hidden/tmp/ckpt/checkpoint.model.keras
9/9 [==============================] - 14s 1s/step - loss: 1.0994 - accuracy: 0.6694 - val_loss: 0.6149 - val_accuracy: 0.7200
Epoch 2/150
9/9 [==============================] - ETA: 0s - loss: 0.6402 - accuracy: 0.6694
Epoch 2: val_accuracy did not improve from 0.72000
9/9 [==============================] - 10s 1s/step - loss: 0.6402 - accuracy: 0.6694 - val_loss: 0.5960 - val_accuracy: 0.7200
Epoch 3/150
9/9 [==============================] - ETA: 0s - loss: 0.6307 - accuracy: 0.6694
Epoch 3: val_accuracy did not improve from 0.72000
9/9 [==============================] - 10s 1s/step - loss: 0.6307 - accuracy: 0.6694 - val_loss: 0.6102 - val_accuracy: 0.7200
Epoch 4/150
9/9 [==============================] - ETA: 0s - loss: 0.6419 - accur

### 1 Hidden layer LSTM With Conv1D

In [ ]:
BATCH_SIZE = 14
EPOCHS = 150

In [ ]:
tunner_1d = RandomSearch(
    build_1d_conv_1lstm,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1
)

In [ ]:
tunner_1d.search(X_train_padded, y_train,
                    epochs=RS_EPOCHS,
                    validation_data=(X_val_padded, y_val)
                    )

In [ ]:
best_hyperparams_1d = tunner_model.get_best_hyperparameters(1)[0]
d1_model = tunner_model.hypermodel.build(best_hyperparams_1d)

In [ ]:
best_hyperparams_1d

In [0]:
d1_model.summary()

In [0]:
checkpoint_1d, early_stop_1d = get_callbacks('asa', 'lstm-1hidden-1dconv')

In [0]:
history_1d = d1_model.fit(x=X_train_padded,
                        y=y_train.reshape((-1,1)),
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=(X_val_padded, y_val.reshape((-1,1))),
                        callbacks=[tensorboard_callback, checkpoint_1d, early_stop_1d])

### 1 Hidden layer LSTM With Conv1D - Includes DropOut

In [13]:
BATCH_SIZE = 14
EPOCHS = 150

In [15]:
tunner_do = RandomSearch(
    build_1d_conv_1layer_lstm_do,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1
)

In [17]:
tunner_do.search(X_train_padded, y_train.reshape((-1,1)),
                    epochs=RS_EPOCHS,
                    validation_data=(X_val_padded,y_val.reshape((-1,1)))
                    )

Trial 10 Complete [00h 01m 26s]
val_accuracy: 0.7196651697158813

Best val_accuracy So Far: 0.7200214862823486
Total elapsed time: 00h 12m 37s


In [18]:
best_hyperparams_do = tunner_do.get_best_hyperparameters(1)[0]
do_model = tunner_do.hypermodel.build(best_hyperparams_do)

In [19]:
do_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 16723, 9)          1737      
                                                                 
 lstm_1 (LSTM)               (None, 16723, 128)        70656     
                                                                 
 dense_1 (Dense)             (None, 16723, 1)          129       
                                                                 
Total params: 72522 (283.29 KB)
Trainable params: 72522 (283.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
checkpoint_do, early_stop_do = get_callbacks('asa', 'lstm-1hidden-1dconv-do')

In [21]:
history_do = do_model.fit(x=X_train_padded,
                        y=y_train.reshape((-1,1)),
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=(X_val_padded, y_val.reshape((-1,1))),
                        callbacks=[tensorboard_callback, checkpoint_do, early_stop_do])

Epoch 1/150
9/9 [==============================] - ETA: 0s - loss: 0.7596 - accuracy: 0.6650
Epoch 1: val_accuracy improved from -inf to 0.71617, saving model to /content/AIForIMUParkinsonData/results/asa/lstm-1hidden-1dconv-do/tmp/ckpt/checkpoint.model.keras
9/9 [==============================] - 10s 828ms/step - loss: 0.7596 - accuracy: 0.6650 - val_loss: 0.6309 - val_accuracy: 0.7162
Epoch 2/150
9/9 [==============================] - ETA: 0s - loss: 0.6515 - accuracy: 0.6689
Epoch 2: val_accuracy improved from 0.71617 to 0.71988, saving model to /content/AIForIMUParkinsonData/results/asa/lstm-1hidden-1dconv-do/tmp/ckpt/checkpoint.model.keras
9/9 [==============================] - 5s 591ms/step - loss: 0.6515 - accuracy: 0.6689 - val_loss: 0.6063 - val_accuracy: 0.7199
Epoch 3/150
9/9 [==============================] - ETA: 0s - loss: 0.6455 - accuracy: 0.6693
Epoch 3: val_accuracy improved from 0.71988 to 0.72000, saving model to /content/AIForIMUParkinsonData/results/asa/lstm-1hidd